In [ ]:
# Load and clean all Google Trends CSV files into one tidy dataset

# 1) Import core libraries
import pandas as pd
import numpy as np
from pathlib import Path

# Define main paths (relative to this notebook in /analysis/)
PROJECT_DIR = Path("..")  # goes one level up (Final Project DSAP/)
GT_DIR = PROJECT_DIR / "datasets" / "raw_data" / "google_trends"

print("Google Trends folder:", GT_DIR.resolve())

Google Trends folder: /files/Final Project DSAP/datasets/raw_data/google_trends


In [2]:
# List all Google Trends CSV files
gt_files = sorted(GT_DIR.glob("gt_*.csv"))

print(f"Number of Google Trends files: {len(gt_files)}")
gt_files[:5]  # show the first 5 paths



Number of Google Trends files: 20


[PosixPath('../datasets/raw_data/google_trends/gt_banking_crisis.csv'),
 PosixPath('../datasets/raw_data/google_trends/gt_bear_market.csv'),
 PosixPath('../datasets/raw_data/google_trends/gt_bull_market.csv'),
 PosixPath('../datasets/raw_data/google_trends/gt_business_confidence.csv'),
 PosixPath('../datasets/raw_data/google_trends/gt_consumer_confidence.csv')]

In [3]:
# Helper function to load and clean a single Google Trends CSV file

def load_single_gt_csv(path: Path) -> pd.DataFrame:
    """
    Load one Google Trends CSV file and return a tidy DataFrame:
    columns = ['Date', 'keyword', 'value'].
    
    Assumes the structure:
    - first line: category info (ignored)
    - second line: blank (ignored)
    - third line: header 'Semaine,<keyword label>'
    - then weekly rows: 'YYYY-MM-DD,<value>'
    """
    # Skip the first two meta lines so that the third line becomes the header
    df = pd.read_csv(path, skiprows=2)
    
    # Rename columns to a standard format
    date_col = df.columns[0]
    value_col = df.columns[1]
    df = df.rename(columns={date_col: "Date", value_col: "value"})
    
    # Parse dates
    df["Date"] = pd.to_datetime(df["Date"])
    
    # Extract a clean keyword name from the file name
    # Example: 'gt_banking_crisis.csv' -> 'banking_crisis'
    keyword = path.stem.replace("gt_", "")
    df["keyword"] = keyword
    
    # Keep only the columns we need
    df = df[["Date", "keyword", "value"]]
    
    return df

# Test on the first file to check the structure
example_df = load_single_gt_csv(gt_files[0])
print(gt_files[0].name)
example_df.head()


gt_banking_crisis.csv


,Date,keyword,value
0,2015-11-01,banking_crisis,60
1,2015-11-08,banking_crisis,47
2,2015-11-15,banking_crisis,49
3,2015-11-22,banking_crisis,46
4,2015-11-29,banking_crisis,71


In [4]:
# Load and concatenate all Google Trends files into one long DataFrame

all_gt_list = []

for path in gt_files:
    df_tmp = load_single_gt_csv(path)
    all_gt_list.append(df_tmp)

gt_long = pd.concat(all_gt_list, ignore_index=True).sort_values(["Date", "keyword"])

print("Shape of long Google Trends data:", gt_long.shape)
gt_long.head()


Shape of long Google Trends data: (5400, 3)


,Date,keyword,value
0,2015-11-01,banking_crisis,60
270,2015-11-01,bear_market,3
540,2015-11-01,bull_market,14
810,2015-11-01,business_confidence,47
1080,2015-11-01,consumer_confidence,34


In [5]:
# Pivot to wide format: one column per keyword, one row per date

gt_wide = gt_long.pivot(index="Date", columns="keyword", values="value").sort_index()

print("Shape of wide Google Trends data:", gt_wide.shape)
gt_wide.head()



Shape of wide Google Trends data: (270, 20)


keyword,banking_crisis,bear_market,bull_market,business_confidence,consumer_confidence,debt_crisis,economic_growth,economic_recession,economic_recovery,financial_crisis,inflation,job_creation,job_losses,low_unemployment,recession,stock_market_crash,stock_market_optimism,stock_market_rally,strong_economy,unemployement
Date,,,,,,,,,,,,,,,,,,,,
2015-11-01,60,3,14,47,34,52,70,29,24,42,71,40,28,14,6,5,0,0,44,5
2015-11-08,47,4,14,38,28,66,65,31,20,48,78,63,25,17,7,5,0,21,44,6
2015-11-15,49,3,15,64,28,62,65,31,25,45,76,43,15,18,7,6,0,0,36,6
2015-11-22,46,2,8,34,26,46,39,17,16,33,62,33,15,9,4,3,0,0,24,5
2015-11-29,71,3,14,61,26,65,69,27,28,48,80,46,14,16,8,6,0,0,41,6


In [6]:
# Check date range
print("Min date:", gt_wide.index.min())
print("Max date:", gt_wide.index.max())

# Save cleaned Google Trends weekly data
CLEAN_DIR = PROJECT_DIR / "datasets" / "clean_data"
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

output_path = CLEAN_DIR / "google_trends_weekly.csv"
gt_wide.to_csv(output_path)

print("Saved cleaned weekly Google Trends data to:", output_path.resolve())


Min date: 2015-11-01 00:00:00
Max date: 2020-12-27 00:00:00
Saved cleaned weekly Google Trends data to: /files/Final Project DSAP/datasets/clean_data/google_trends_weekly.csv
